In [4]:
!conda install -c conda-forge -y --force-reinstall python==3.11.5
!pip install --extra-index-url https://pypi.nvidia.com cudf-cu12
!pip install --extra-index-url https://pypi.nvidia.com cuml-cu12
!pip install --extra-index-url https://pypi.nvidia.com cugraph-cu12

Channels:
 - conda-forge
 - defaults
 - rapidsai
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/opc/miniconda3/envs/rapids-24.06

  added / updated specs:
    - python==3.11.5





Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
from collections import OrderedDict
import gc
import glob
import os
import pickle
import re
import sys
import time

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


n_gpus = 1
isPlot = False

if not n_gpus:
    from pandas import read_csv, concat, merge, DataFrame, get_dummies, Series, isnull
    from sklearn.preprocessing import OneHotEncoder
    from numpy import zeros, double, int8, concatenate
else:
    from cudf import read_csv, concat, merge, DataFrame, get_dummies, Series
    from cuml.preprocessing import OneHotEncoder
    from cupy import zeros,double, int8, concatenate

In [2]:
col_acq = ['LoanID', 'Channel', 'SellerName', 'OrInterestRate', 'OrUnpaidPrinc', 'OrLoanTerm',
           'OrDate', 'FirstPayment', 'OrLTV', 'OrCLTV', 'NumBorrow', 'DTIRat', 'CreditScore',
           'FTHomeBuyer', 'LoanPurpose', 'PropertyType', 'NumUnits', 'OccStatus', 'PropertyState',
           'Zip', 'MortInsPerc', 'ProductType', 'CoCreditScore', 'Extra', 'MortInsType', 'RelMortInd'];
dtypesDict = OrderedDict([
    ("LoanID", "int64"),
    ("Channel", "category"),
    ("SellerName", "category"),
    ("OrInterestRate", "float64"),
    ("OrUnpaidPrinc", "int64"),
    ("OrLoanTerm", "int64"),
    ("OrDate", "str"),
    ("FirstPayment", "str"),
    ("OrLTV", "int64"),  # was:float64
    ("OrCLTV", "float64"),
    ("NumBorrow", "float64"),
    ("DTIRat", "float64"),
    ("CreditScore", "float64"),
    ("FTHomeBuyer", "category"),
    ("LoanPurpose", "category"),
    ("PropertyType", "category"),
    ("NumUnits", "float64"),
    ("OccStatus", "category"),
    ("PropertyState", "category"),
    ("Zip", "int64"),
    ("MortInsPerc", "float64"),
    ("ProductType", "category"),
    ("CoCreditScore", "float64"),
    ("Extra", "int64"),
    ("MortInsType", "category"),
    ("RelMortInd", "category"),
])

dtype = list(dtypesDict.values())
print(dtype)
print('col_acq list is len', len(col_acq))

['int64', 'category', 'category', 'float64', 'int64', 'int64', 'str', 'str', 'int64', 'float64', 'float64', 'float64', 'float64', 'category', 'category', 'category', 'float64', 'category', 'category', 'int64', 'float64', 'category', 'float64', 'int64', 'category', 'category']
col_acq list is len 26


In [3]:
# open Acquisition files from each quarter 2007 - 2012

import os,oci,ocifs
from oci.config import from_file
from oci.config import validate_config

config = from_file(file_location="~/.oci/config")
validate_config(config)
object_storage_client = oci.object_storage.ObjectStorageClient(config)
pattern = r'Acquisition_(20[0][7,8,9]|201[0,1,2])Q[1-4].txt'
acq_object_list = object_storage_client.list_objects(namespace_name="${bucket_namespace}", bucket_name='mortgage_bucket_dataset', prefix='acq')

acqs = []

for o in acq_object_list.data.objects:
    acqs.append(o.name.replace("acq/",""))
fpaths = sorted(filter(re.compile(pattern).match, list(filter(None, acqs))))
print(len(fpaths))
fs = ocifs.OCIFileSystem(config)
v_bucket_name = 'mortgage-bucket-dataset'
src_namespace = 'orasenatdpltintegration03'

df_acq = concat([read_csv(fs.open(f'{v_bucket_name}@{src_namespace}/acq/{fp}','rb'), 
                          sep='|', names=col_acq, index_col=False) for fp in fpaths], 
                axis=0).reset_index(drop=True)

print('\n'.join(fpaths))
df_acq.head()


24
Acquisition_2007Q1.txt
Acquisition_2007Q2.txt
Acquisition_2007Q3.txt
Acquisition_2007Q4.txt
Acquisition_2008Q1.txt
Acquisition_2008Q2.txt
Acquisition_2008Q3.txt
Acquisition_2008Q4.txt
Acquisition_2009Q1.txt
Acquisition_2009Q2.txt
Acquisition_2009Q3.txt
Acquisition_2009Q4.txt
Acquisition_2010Q1.txt
Acquisition_2010Q2.txt
Acquisition_2010Q3.txt
Acquisition_2010Q4.txt
Acquisition_2011Q1.txt
Acquisition_2011Q2.txt
Acquisition_2011Q3.txt
Acquisition_2011Q4.txt
Acquisition_2012Q1.txt
Acquisition_2012Q2.txt
Acquisition_2012Q3.txt
Acquisition_2012Q4.txt


,LoanID,Channel,SellerName,OrInterestRate,OrUnpaidPrinc,OrLoanTerm,OrDate,FirstPayment,OrLTV,OrCLTV,NumBorrow,DTIRat,CreditScore,FTHomeBuyer,LoanPurpose,PropertyType,NumUnits,OccStatus,PropertyState,Zip,MortInsPerc,ProductType,CoCreditScore,Extra,MortInsType,RelMortInd
0,100001461640,R,"PNC BANK, N.A.",6.250,137000,360,01/2007,03/2007,56,56,2,37,741,N,C,SF,1,P,MI,486,<NA>,FRM,734,<NA>,N,200701
1,100015135004,R,SUNTRUST MORTGAGE INC.,6.000,116000,360,02/2007,04/2007,80,80,2,11,796,N,R,SF,1,S,GA,302,<NA>,FRM,762,<NA>,N,200701
2,100015306566,C,"CITIMORTGAGE, INC.",6.375,58000,180,02/2007,03/2007,78,78,2,30,710,N,R,SF,1,P,IN,465,<NA>,FRM,<NA>,<NA>,N,200701
3,100015319835,C,"BANK OF AMERICA, N.A.",6.125,353000,360,12/2006,02/2007,80,80,2,28,778,N,R,SF,1,P,MA,21,<NA>,FRM,656,<NA>,N,200701
4,100030521552,C,"GMAC MORTGAGE, LLC",5.875,385000,360,12/2006,03/2007,70,70,2,50,720,N,C,SF,1,P,CA,917,<NA>,FRM,700,<NA>,N,200701


In [4]:
col_per = ['LoanID', 'MonthRep', 'Servicer', 'CurrInterestRate', 'CAUPB', 'LoanAge', 'MonthsToMaturity',
           'AdMonthsToMaturity', 'MaturityDate', 'MSA', 'CLDS', 'ModFlag', 'ZeroBalCode', 'ZeroBalDate',
           'LastInstallDate', 'ForeclosureDate', 'DispositionDate', 'ForeclosureCosts', 'PPRC', 'AssetRecCost', 'MHRC',
           'ATFHP', 'NetSaleProceeds', 'CreditEnhProceeds', 'RPMWP', 'OFP', 'NIBUPB', 'PFUPB', 'RMWPF',
           'FPWA', 'ServicingIndicator']
print('col_per list is len',len(col_per),'but using only 2!')
start = time.time()
perf_object_list = object_storage_client.list_objects(namespace_name="${bucket_namespace}", bucket_name='mortgage-bucket-dataset', prefix='perf')

perfs = []
for o in perf_object_list.data.objects:
    perfs.append(o.name.replace("perf/",""))

for fp in sorted(list(filter(None, perfs)))[:2]:
    print(f'{v_bucket_name}@{src_namespace}/perf/{fp}')
print(len(sorted(list(filter(None, perfs)))[:2]))
df_per = concat([read_csv(fs.open(f'{v_bucket_name}@{src_namespace}/perf/{fp}','rb'), 
                          sep='|', names=col_per, usecols=[0, 10], dtype={'LoanID': int, 'CLDS': str}, index_col=False) 
                 for fp in sorted(list(filter(None, perfs)))[:2]], axis=0)

print(time.time()-start,'secs')
tread = time.time()-start
print(df_per.shape)
df_per.loc[df_per['LoanID'] == 100006270289].tail(10) #,100960628290]].tail(10)

col_per list is len 31 but using only 2!
mortgage-bucket-dataset@orasenatdpltintegration03/perf/Performance_2000Q1.txt
mortgage-bucket-dataset@orasenatdpltintegration03/perf/Performance_2000Q2.txt
2
18.820766925811768 secs
(17302760, 2)


,LoanID,CLDS


In [6]:
print(df_per.shape)
df_per = df_per.dropna(subset=['CLDS'])
df_per['CLDS'] = df_per['CLDS'].astype(str)
print(df_per.shape)
df_per.loc[df_per['LoanID'] == 100006270289].tail(10) #100960628290].tail(10)

(17302760, 2)
(17302760, 2)


,LoanID,CLDS


In [7]:
### SET TO 4 MONTHS (str) below.###
NUM_MONTHS = '4'
###################################


df_delinq4 = df_per.loc[df_per['CLDS'] == NUM_MONTHS]

print(df_delinq4.shape)
df_delinq4.drop_duplicates(subset='LoanID', keep='last', inplace=True)
print(df_delinq4.shape)
df_delinq4.loc[df_delinq4['LoanID'] == 100006270289].tail(10)

(29342, 2)
(15718, 2)


,LoanID,CLDS


In [9]:
print(df_acq.shape, df_delinq4.shape)

start = time.time()
df = merge(df_acq, df_delinq4, on='LoanID', how='outer')
df['Zip'] = df['Zip'].astype(str)
print(time.time()-start, 'secs')

del df_acq, df_per, df_delinq4
gc.collect()

df = df.reset_index().rename(columns={'CLDS': 'Default'})
print(df.shape)

df.loc[df['Default'] == '4', 'Default'] = 1
df.loc[df['Default'].isnull(), 'Default'] = 0  # for null caused by outer join

df['Default'] = df['Default'].astype(int)
df.loc[df['LoanID'] == 100006270289].tail(10)  # Yes missed payments and so w/100960628290

(11232359, 26) (15718, 2)
0.1619861125946045 secs
(11248077, 28)


,index,LoanID,Channel,SellerName,OrInterestRate,OrUnpaidPrinc,OrLoanTerm,OrDate,FirstPayment,OrLTV,OrCLTV,NumBorrow,DTIRat,CreditScore,FTHomeBuyer,LoanPurpose,PropertyType,NumUnits,OccStatus,PropertyState,Zip,MortInsPerc,ProductType,CoCreditScore,Extra,MortInsType,RelMortInd,Default
1249651,1249651,100006270289,C,"WELLS FARGO BANK, N.A.",7.5,226000,360,11/2007,01/2008,95,95,1,42,704,Y,P,SF,1,P,CA,933,30,FRM,<NA>,1,N,200801,0


In [10]:
df['Default'].value_counts()
#Test no default  case:
df.loc[df['LoanID'] == 100010079393]

,index,LoanID,Channel,SellerName,OrInterestRate,OrUnpaidPrinc,OrLoanTerm,OrDate,FirstPayment,OrLTV,OrCLTV,NumBorrow,DTIRat,CreditScore,FTHomeBuyer,LoanPurpose,PropertyType,NumUnits,OccStatus,PropertyState,Zip,MortInsPerc,ProductType,CoCreditScore,Extra,MortInsType,RelMortInd,Default
5090177,5090177,100010079393,C,"WELLS FARGO BANK, N.A.",4.875,284000,360,01/2010,03/2010,80,80,1,32,773,Y,P,PU,1,P,TX,787,<NA>,FRM,<NA>,<NA>,N,201001,0


In [11]:
df.loc[df['Default'] == 1].head(10)  # focus on rearer defaults

,index,LoanID,Channel,SellerName,OrInterestRate,OrUnpaidPrinc,OrLoanTerm,OrDate,FirstPayment,OrLTV,OrCLTV,NumBorrow,DTIRat,CreditScore,FTHomeBuyer,LoanPurpose,PropertyType,NumUnits,OccStatus,PropertyState,Zip,MortInsPerc,ProductType,CoCreditScore,Extra,MortInsType,RelMortInd,Default
11232359,11232359,100023274028,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232360,11232360,100289940808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232361,11232361,100366510395,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232362,11232362,100416816452,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232363,11232363,100557946796,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232364,11232364,100579898395,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232365,11232365,100654236473,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232366,11232366,100687390672,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232367,11232367,100710809124,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
11232368,11232368,100993568697,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1


In [12]:
nullHistogram = None
if n_gpus == 0:
    nullHistogram = df.apply(lambda x: x.isnull().sum(), axis=0)
nullHistogram

In [13]:
if n_gpus == 0 and isPlot:
    sns.countplot(df['Default'])
    vc = df['Default'].value_counts()  # Let's see the imbalance 
    print('default density = ', vc[1]/vc[0], "or 1 in", np.round(vc[0]/vc[1],1))
    vc

In [14]:
if n_gpus == 0 and isPlot:
    columns = ['OrInterestRate','OrCLTV','DTIRat','CreditScore','OrUnpaidPrinc']
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(8,7))
    plt.tight_layout(w_pad=3.0, h_pad=3.0)

    print(range(1, len(columns)+1), columns)
    for i, column in zip(range(1, len(columns)+1), columns):
        plt.subplot(3, 3, i)
        print(i,column)
        print(len(df["Default"]));print(len(df[column]))
        sns.boxplot(x="Default", y=column, data=df, linewidth=0.5)
        plt.xlabel('Default')

In [15]:
df.columns

Index(['index', 'LoanID', 'Channel', 'SellerName', 'OrInterestRate', 'OrUnpaidPrinc', 'OrLoanTerm', 'OrDate', 'FirstPayment', 'OrLTV', 'OrCLTV', 'NumBorrow', 'DTIRat', 'CreditScore', 'FTHomeBuyer', 'LoanPurpose', 'PropertyType', 'NumUnits', 'OccStatus', 'PropertyState', 'Zip', 'MortInsPerc', 'ProductType', 'CoCreditScore', 'Extra', 'MortInsType', 'RelMortInd', 'Default'], dtype='object')

In [16]:
df.drop(['index','OrDate','OrLTV','MortInsPerc','RelMortInd','FirstPayment'], axis=1, inplace=True)

print(df.shape)

(11248077, 22)


In [17]:
def fillnan(df):
    if n_gpus == 0: 
        columns = df.columns[df.isnull().any().tolist()]
    else: 
        columns = df.columns[df.isnull().any().to_arrow().to_pylist()]
    for name in columns:
        print(name)
        if df[name].dtype == 'object':
            df.loc[df[name].isnull(), name] = df[name].mode().to_pandas()[0]
        else:
            #  a possible better way could be to groupby the particular time period and use those values
            #  ex. the interest rate during a particular month, year, state combination.
            df.loc[df[name].isnull(), name] = df[name].mean()
    return df
df = fillnan(df)

Channel
SellerName
OrInterestRate
OrUnpaidPrinc
OrLoanTerm
OrCLTV
NumBorrow
DTIRat
CreditScore
FTHomeBuyer
LoanPurpose
PropertyType


/home/opc/miniconda3/lib/python3.11/site-packages/cudf/core/series.py:264: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '216744.60253629714' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  warnings.warn(
/home/opc/miniconda3/lib/python3.11/site-packages/cudf/core/series.py:264: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '306.7032233389264' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  warnings.warn(
/home/opc/miniconda3/lib/python3.11/site-packages/cudf/core/series.py:264: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '69.21542575023643' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  warnings.warn(
/home/opc/miniconda3/lib/python3.11/site-packages/c

NumUnits
OccStatus
PropertyState
Zip
ProductType
CoCreditScore
Extra
MortInsType


/home/opc/miniconda3/lib/python3.11/site-packages/cudf/core/series.py:264: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '766.8680345019693' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  warnings.warn(
/home/opc/miniconda3/lib/python3.11/site-packages/cudf/core/series.py:264: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0775547418592215' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  warnings.warn(


In [18]:
df['Channel'], Channel = df['Channel'].factorize()
df['SellerName'], SellerName = df['SellerName'].factorize()
df['FTHomeBuyer'], FTHomeBuyer = df['FTHomeBuyer'].factorize()
df['LoanPurpose'], LoanPurpose = df['LoanPurpose'].factorize()
df['PropertyType'], PropertyType = df['PropertyType'].factorize()
df['OccStatus'], OccStatus = df['OccStatus'].factorize()
df['PropertyState'], PropertyState = df['PropertyState'].factorize()
df['Zip'], Zip = df['Zip'].factorize()
df['ProductType'], ProductType = df['ProductType'].factorize()
df['MortInsType'], MortInsType = df['MortInsType'].factorize()

In [19]:
# save cleaned data to parquet file

os.makedirs('./rapids/notebooks/cleaned_data', exist_ok=True)
df.to_parquet('./rapids/notebooks/cleaned_data/fnma_cleaned.pq')

In [20]:
# save these mappings for transforming back and forth between factorized and real values
mappings = {
    'Channel': Channel.to_pandas().tolist(),
    'SellerName': SellerName.to_pandas().tolist(),
    'FTHomeBuyer': FTHomeBuyer.to_pandas().tolist(),
    'LoanPurpose': LoanPurpose.to_pandas().tolist(),
    'PropertyType': PropertyType.to_pandas().tolist(),
    'OccStatus': OccStatus.to_pandas().tolist(),
    'PropertyState': PropertyState.to_pandas().tolist(),
    'Zip': Zip.to_pandas().tolist(),
    'ProductType': ProductType.to_pandas().tolist(),
    'MortInsType': MortInsType.to_pandas().tolist(),
}

In [21]:
with open('./rapids/notebooks/cleaned_data/fnma_mappings.pkl', 'wb') as f:
    pickle.dump(mappings, f)

# #  test opening pickle file to make sure items serialized appropriately
# with open('fnma_mappings.pkl', 'rb') as f:
#     asdf = pickle.load(f)